# Objective:	To	implement	and	experiment	with	Monte	Carlo	Tree	Search as	a	method	for	planning/strategic	reasoning,	using	Connect	Four	as	a	sample	domain.

In [ ]:
from google.colab import files
files.upload()

In [4]:
import sys
import random
import numpy as np
import copy
import math
import time
from collections import defaultdict

Rules for the AKQ Game (Clarivoyance Game):


1.   both players ante 1 dollar
2.   OOP (out of position) can bet 1 dollar or check
3. OOP always has a K
4. facing a bet IP (in position) can call or fold
5. if OOP did not bet IP can bet 1 dollar or check
6. after all actions are done whoever has the best card wins the pot




In [ ]:
 #We can represent the game state liike this
 #pot ,OOP actions ,OOP holding ,IP actions ,IN position Holdings
 [pot,[check, bet], 2, [check, bet], 3 'OOP']

 #1 represents a Q
 #2 represents a K
 #3 represents a A

In [35]:
class MonteCarloTreeSearchNode():
  """
  state: For our game it represents the board state.
  parent: It is None for the root node and for other nodes it is equal to the node it is derived from. For the first turn as you have seen from the game it is None.
  children: It contains all possible actions from the current node.
  parent_action: None for the root node and for other nodes it is equal to the action which it’s parent carried out.
  _number_of_visits: Number of times current node is visited
  results: It’s a dictionary
  _untried_actions: Represents the list of all possible actions
  action: Move which has to be carried out.
  """
  def __init__(self, state, parent=None, parent_action=None, player = 'IP'):
    self.state = state
    self.parent = parent
    self.player = player
    self.parent_action = parent_action
    self.verbosity = 0
    self.children = []
    self._number_of_visits = 0
    self._results = defaultdict(int)
    self._results[1] = 0
    self._results[-1] = 0
    self._untried_actions = None
    self._untried_actions = self.untried_actions()
    return

  def untried_actions(self):
    """
    get any possible actions
    """
    self._untried_actions = self.get_legal_actions()
    return self._untried_actions

  def q(self):
      wins = self._results[1]
      losses = self._results[-1]
      return wins - losses

  def n(self):
      return self._number_of_visits

  def UCB(self):
    if self.n() == 0: return math.sqrt(2)
    return self._results[1] / self.n() + math.sqrt(math.log(self.parent.n())/self.n())

  def expand(self):
      """
      From the present state, next state is generated depending on the action which is carried out.
      In this step all the possible child nodes corresponding to generated states are appended to the children array and the child_node is returned.
      The states which are possible from the present state are all generated and the child_node corresponding to this generated state is returned.
      """
      untried_actions = copy.deepcopy(self._untried_actions)
      while len(untried_actions) != 0:
        action = untried_actions.pop()
        next_state = self.move(action)
        child_node = MonteCarloTreeSearchNode(next_state, parent=self, parent_action=action)
        self.children.append(child_node)
      print(self.children)

  def is_terminal_node(self):
      """
      This is used to check if the current node is terminal or not.
      Terminal node is reached when the game is over.
      """
      return self.is_game_over()

  def rollout(self):
      """
      From the current state, entire game is simulated till there is an outcome for the game.
      the outcome of the game is returned
      """
      print('rolling out')
      current_node = copy.deepcopy(self)
      print(current_node.state)
      while not current_node.is_game_over():
        possible_moves = current_node.get_legal_actions()
        action = current_node.rollout_policy(possible_moves)#later its possible to change how we select nodes for simulation
        print('Move Selected:', action)
        current_node.state = current_node.move(action)
        print(current_node.state)

      return current_node.game_result()

  def backpropagate(self, result):
      """
      In this step all the statistics for the nodes are updated.
      """
      if self.verbosity == 2:
        print('backpropogating...')
      self._number_of_visits += 1.
      self._results[result] += 1.
      if self.parent:
        if self.verbosity == 2:
          print('New Values:')
          print('Wi:', self._results[1])
          print('Ni:', self.n())
        self.parent.backpropagate(result)

  def is_fully_expanded(self):
      """
      returns true if all actions are popped from untried actions
      """
      return len(self._untried_actions) == 0

  def best_child(self, c_param=0.1):
      """
      Once fully expanded, this function selects the best child out of the children array.
      """
      possible_children = []
      for child in self.children:
        if child.n() > 0:
          possible_children.append(child)

      choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in possible_children]
      return self.children[np.argmax(choices_weights)]

  def rollout_policy(self, possible_moves):
      """
      randomly selects a move out of possible moves
      """
      return possible_moves[np.random.randint(len(possible_moves))]

  def _tree_policy(self, algo = 'MCTS'):
      """
      Selects node to run rollout.
      """
      if algo == 'MCTS':
        current_node = self
        while len(current_node.children) > 0 : #while node is not leaf
          children = current_node.children
          current_node = random.choice(children)
        return current_node

      elif algo == 'UCT':
        current_node = self
        while len(current_node.children) > 0: # while node is not a leaf
          max_node = None
          max_value= float('-inf')
          possible_children = []

          if current_node.verbosity == 2:
            print('Wi : ' , current_node._results[1])
            print('Ni : ' , current_node.n())

          for node in current_node.children:
            UCB = node.UCB()
            if node.verbosity == 2:
              print('Child UCB Value:' , UCB)

            if UCB > max_value:
              max_value = UCB

          for child in current_node.children:
            if child.UCB() == max_value:
              possible_children.append(child)

          current_node = random.choice(possible_children)
          if current_node.verbosity == 2:
            print('Move Selected:', current_node.parent_action)

        return current_node


  def best_action(self, simulations = 100, c_param=0.1, algo = 'MCTS'):
      """
      This is the best action function which returns the node corresponding to best possible move.
      The step of expansion, simulation and backpropagation are carried out by the code above.
      """
      start_time = time.time()

      for i in range(simulations):
        v = self._tree_policy(algo)#select node based on tree policy
        v.expand()
        reward = v.rollout()
        v.backpropagate(reward)

      if self.verbosity == 1 or self.verbosity == 2:
        print('Number of rollouts:', simulations)
        print('Seconds elapsed: ', time.time()- start_time)

      return self.best_child(c_param)

  def is_game_over(self):
    """
    Returns true or false depending on if game is over
    """
    #check check
    if len(self.state[1]) + len(self.state[3]) == 0:
      return True

    #check bet
    if len(self.state[3]) == 0:
      if len(self.state[1]) == 1:
        return True

    # bet call/fold
    if len(self.state[1]) == 0:
      if len(self.state[3]) == 1:
        return True

    return False



  def game_result(self):
    """
    Returns 1, 0, or -1 depending on the game state:
    1 for a win, 0 for a tie, and -1 for a loss.

    this function might be modified to return the pot size instead of a win
    """
    def get_winner():
      if self.state[2] < self.state[4]:
        return self.state[0]
      return self.state[0]//2 * -1

    #check check
    if len(self.state[1]) + len(self.state[3]) == 0:
      return get_winner()

    #check bet call/fold
    if len(self.state[3]) == 0:
      if 'fold' in self.state[1]:
        #means OOP called
        return get_winner()
      else:
        #means OOP folded
        return self.state[0]

    #bet call/fold
    if len(self.state[1]) == 0:
      if 'fold' in self.state[3]:
        #means IP called
        return get_winner()
      else:
        #means IP folded
        return self.state[0]

  def move(self, action):
      """
      Given an action, Return a new state after that action occurs
      """
      #[pot, [check,bet], 2, [check,bet] ,3, 'OOP']
      def flip_turn(state):
        player = state[-1]
        if player == 'OOP':
          state[-1] = 'IP'
          return state
        state[-1] = 'OOP'
        return state

      player = self.state[-1]
      state = copy.deepcopy(self.state)

      if action == 'check':
        if player == 'OOP':
          state[1] = []
        if player == 'IP':
          state[3] = []

      if action == 'bet':
        state[0] = state[0] + 1
        if player == 'OOP':
          state[1] = []
          state[3] = ['call', 'fold']

        if player == 'IP':
          state[3] = []
          state[1] = ['call', 'fold']

      if action == 'call':
        state[0] = state[0] + 1

        if player == 'OOP':
          state[1].remove(action)
        if player == 'IP':
          state[3].remove(action)

      if action == 'fold':

        if player == 'OOP':
          state[1].remove(action)

        if player == 'IP':
          state[3].remove(action)

      return flip_turn(state)


  def get_legal_actions(self):
      '''
      gets legal moves of the state
      '''
      if self.state[-1] == 'OOP':
        return self.state[1]
      return self.state[3]


In [52]:
def main(verbosity, simulations):

    #for testing
    algorithm = 'PMCGS'

    state = [2, [], 2, ['check', 'bet'], 3, 'IP']

    if algorithm == 'PMCGS':
      root = MonteCarloTreeSearchNode(state = state)
      selected_node = root.best_action(simulations = simulations)
      print(selected_node.parent_action)

    if algorithm == 'UCT':
      root = MonteCarloTreeSearchNode(state = state)
      selected_node = root.best_action(simulations = simulations, algo='UCT')
      print(selected_node.parent_action)


if __name__ == "__main__":
    verbosity = 'Verbose'  # Verbose, Brief, or None
    simulations = 100  # Number of simulations, not used for UR but required for the script

    main(verbosity ,simulations)


[<__main__.MonteCarloTreeSearchNode object at 0x7be3df9c0130>, <__main__.MonteCarloTreeSearchNode object at 0x7be3df9c2380>]
rolling out
[2, [], 2, ['check', 'bet'], 3, 'IP']
Move Selected: bet
[3, ['call', 'fold'], 2, [], 3, 'OOP']
Move Selected: fold
[3, ['call'], 2, [], 3, 'IP']
[<__main__.MonteCarloTreeSearchNode object at 0x7be3df9c21a0>, <__main__.MonteCarloTreeSearchNode object at 0x7be3df9c1c00>]
rolling out
[3, ['call', 'fold'], 2, [], 3, 'OOP']
Move Selected: call
[4, ['fold'], 2, [], 3, 'IP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[3, ['call'], 2, [], 3, 'IP']
[]
rolling out
[3, ['call'], 2, [], 3, 'IP']
[]
rolling out
[4, ['fold'], 2, [], 3, 'IP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[3, ['call'], 2, [], 3, 'IP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[4, ['fold'], 2, [], 3, 'IP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[2, [], 2, [], 3, 'OOP']
[]
rolling out
[2, [], 2, 